In [ ]:
import daft
from daft import col, DataType as dt
import numpy as np
import jax
import jax.numpy as jnp
from jax.extend import backend
import tensorflow as tf
from videoprism import models as vp

tf.config.set_visible_devices([], "GPU")
tf.config.set_visible_devices([], "TPU")

In [ ]:
PATHS = ["/Users/everett-founder/Movies/digitlism.mp4"]
NUM_FRAMES = 16# VideoPrism requires 16x288x288 frames
FRAME_SIZE = 288
MODEL_NAME = 'videoprism_lvt_public_v1_base'

In [ ]:
df_frames = daft.read_video_frames(
    PATHS,
    image_height=288, 
    image_width=288,
)